<a href="https://colab.research.google.com/github/YoTaSo/datascience/blob/deeplearning/mymainpytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.autograd import Function
from torch.autograd import gradcheck
from torch.utils.tensorboard import SummaryWriter

In [ ]:
url='https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data'
df1 = pd.read_csv(url,delim_whitespace=True,header=None)
xMat=df1.iloc[:,0:13]
yRes=df1.iloc[:,13]

In [ ]:
augX=[]
for i in range(xMat.shape[0]):
  b=np.concatenate([[1],xMat.values[i]])
  augX.append(b)
augX = np.asarray(augX)

In [ ]:
#use GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
x_train_tensor = torch.from_numpy(augX).float().to(device)
y_train_tensor = torch.from_numpy(yRes.values).float().to(device)

In [ ]:
y_train_tensor = y_train_tensor.view(-1,1)

In [ ]:
w = torch.randn(1,14,requires_grad=True,dtype=torch.float,device=device)

In [ ]:
#to check every thing is well defined
#x_train_tensor.mm(w)
torch.mm(x_train_tensor,w.t())

In [ ]:
class Context:
    """Very simplified context object"""
    def __init__(self):
        self._saved_tensors = ()
    def save_for_backward(self, *args):
        self._saved_tensors = args
    @property
    def saved_tensors(self):
        return self._saved_tensors

In [ ]:


class MaFonction(Function):
  @staticmethod
  def forward(ctx, x,w):
    ctx.save_for_backward(x,w)
    return x.mm(w.t())
    
  @staticmethod
  def backward(ctx, grad_output):
    x,w = ctx.saved_tensors
    grad_x = grad_w = None
    grad_x = grad_output.mm(w)
    grad_w = grad_output.t().mm(x)
    #grad_b = grad_output.sum(0).squeeze(0)

    return grad_x, grad_w

In [ ]:
class MSE(Function):
    @staticmethod
    def forward(ctx, input, response):
        ctx.save_for_backward(input, response)
        output = (input - response)**2
        return output

    @staticmethod
    def backward(ctx,grad_output):
        input, response = ctx.saved_tensors
        g_inp = grad_output*(2*(input-response))
        #g_inp = 2*(input-response)
        #g_res = 2*(response-input)
        g_res = grad_output*(2*(response-input))
        return g_inp, g_res


In [ ]:
mafonction = MaFonction()
mse = MSE()
ctx = Context()
ctx_mse = Context()
output = mafonction.forward(ctx,x_train_tensor,w)
#mafonction_grad = mafonction.backward(ctx,1)
#print(output)

In [ ]:

mafonction_check = MaFonction.apply
x = torch.randn(506,14,requires_grad=True,dtype=torch.float64,device=device)
w = torch.randn(1,14,requires_grad=True,dtype=torch.float64,device=device)
#b = torch.randn(1,requires_grad=True,dtype=torch.float64,device=device)
torch.autograd.gradcheck(mafonction_check,(x,w))
#torch.autograd.gradcheck()

True

In [ ]:
mafonction_check = MSE.apply

x = torch.randn(506,14,requires_grad=True,dtype=torch.float64 ,device=device)
w = torch.randn(1,14,requires_grad=True,dtype=torch.float64 ,device=device)

#

y = torch.randn(506,1,requires_grad=True,dtype=torch.float64 ,device=device)
yhat=torch.randn(506,1,requires_grad=True,dtype=torch.float64 ,device=device)
#yhat = mafonction.forward(ctx,x_train_tensor,w)
#b = torch.randn(1,requires_grad=True,dtype=torch.float64,device=device)
torch.autograd.gradcheck(mafonction_check,(y, yhat))

#outForwTest=mse.forward(ctx_mse, output, y_train_tensor)
#y_grad, _ =mse.backward(ctx_mse, torch.ones(506, 1, requires_grad=True, dtype=torch.float ,device=device))
#_, grad_w= mafonction.backward(ctx, y_grad)

True

In [ ]:
y_train_tensor.size()

torch.Size([506, 1])

In [ ]:
torch.mean(outForwTest).item()

70211.7890625

In [ ]:
#batch learning
torch.random.manual_seed(3360)
w = torch.randn(1,14,requires_grad=True,dtype=torch.float,device=device)
learning_rate = 1e-6
for t in range(10000):
  y_pred = mafonction.forward(ctx,x_train_tensor,w)
  loss = (y_pred - y_train_tensor).pow(2).mean()
  if t%50==0:
    print(t, loss.item())

  loss.backward()
  
  with torch.no_grad():
    w -= learning_rate * w.grad

  w.grad.zero_()

In [ ]:
#batch learning
torch.random.manual_seed(3360)
w = torch.randn(1,14,requires_grad=True,dtype=torch.float,device=device)
learning_rate = 1e-9
for t in range(10000):
  y_pred = mafonction.forward(ctx,x_train_tensor,w)
  loss = mse.forward(ctx_mse, y_pred, y_train_tensor)
  
  if t%50==0:
    meanLoss=torch.mean(loss).item()
    print(t, meanLoss)
  y_grad, _ = mse.backward(ctx_mse,torch.ones(506, 1, requires_grad=True, dtype=torch.float ,device=device))
  _, grad_w= mafonction.backward(ctx, y_grad)

  w = w - learning_rate*grad_w

In [ ]:
#Stochastic gradient decent
learning_rate = 1e-6
for t in range(500):

  mylinear = MaFonction.apply
  #y_pred = mafonction.forward(ctx,x_train_tensor,w)
  #randInd=np.random.randint(x_train_tensor.size()[0])
  randInd=torch.randint(x_train_tensor.size()[0],size=(1,))

  sampled_tensor, sampled_y_train=x_train_tensor[randInd],y_train_tensor[randInd]
  sampled_tensor = sampled_tensor.view(-1,1)
  sampled_tensor=sampled_tensor.t()

  y_pred=mylinear(sampled_tensor,w)
  
  loss = (y_pred - y_train_tensor).pow(2).mean()
  print(t, loss.item())

  loss.backward()

  with torch.no_grad():
    w -= learning_rate * w.grad

    w.grad.zero_()

In [ ]:
batchSize = 50 # or whatever

for t in range(500):

  mylinear = MaFonction.apply
  #y_pred = mafonction.forward(ctx,x_train_tensor,w)
  trani_tes_len=x_train_tensor.size()[0]
  perms = torch.randperm(trani_tes_len)
  for i in range(0,trani_tes_len, batchSize):

    indices = perms[i:i+batchSize]
    batch_x_train, batch_y_train = x_train_tensor[indices], y_train_tensor[indices]
    #sampled_tensor = sampled_tensor.view(-1,1)
    #sampled_tensor=sampled_tensor.t()
    y_pred_batch=mylinear(batch_x_train,w)

    loss = (y_pred_batch - batch_y_train).pow(2).mean()
    print(t,i, loss.item())
    loss.backward()

    with torch.no_grad():
      w -= learning_rate * w.grad
      w.grad.zero_()


In [ ]:
train_tes_len=x_train_tensor.size()[0]
perms = torch.randperm(train_tes_len)
breakInd=int(np.floor(train_tes_len*.7))# 70 30 split

In [ ]:
train_indx=perms[:breakInd]
test_indx=perms[breakInd:]

In [ ]:
x_train, y_train = x_train_tensor[train_indx], y_train_tensor[train_indx]
x_test, y_test = x_train_tensor[test_indx], y_train_tensor[test_indx]

In [ ]:


writer = SummaryWriter()
w = torch.randn(1,14,requires_grad=True,dtype=torch.float,device=device)
learning_rate = 1e-6
for t in range(5000):
  mylinear = MaFonction.apply
  #y_pred = mafonction.forward(ctx,x_train_tensor,w)
  y_pred_train=mylinear(x_train,w)
  loss = (y_pred_train - y_train).pow(2).mean()
  #print(t, loss.item())
  y_pred_test=mylinear(x_test,w)
  lossTest = (y_pred_test - y_test).pow(2).mean()

  #writer.add_scalar('Loss/trainTest', loss.item(), t)
  #writer.add_scalar('Loss/trainTest', lossTest.item(), t)
  
  writer.add_scalars('Loss/compare2', {'Train':loss.item(),'Test':lossTest.item()}, t)

  loss.backward()

  with torch.no_grad():
    w -= learning_rate * w.grad

    w.grad.zero_()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=runs